# Compute monomials and relations from multiplication table in A(9)

In [1]:
#Load .txt file which contains multiplication table of A(9)
file1 = open('multiplicationtableA9.txt', 'r') 
Lines = file1.readlines()

In [2]:
#Define function for reading the specific lines of the file
def read_string(s):
    L = s.split()
    
    g = L[2]
    values = {}
    
    for a in L[4:]:
        if a == "+":
            pass
        else:
            x = a.split("*")
            values[x[1]] = int(x[0])
    return g, values

In [3]:
# collect multiplication tables from (x,y,z,w) = (g_2, g_2^2, g_2^3, g_2^4)
xtable = {}
ytable = {}
ztable = {}
wtable = {}

for line in Lines: 
    if line[0:5] == "g_2^1":
        key,val = read_string(line)
        xtable[key] = val
    if line[0:5] == "g_2^2":
        key,val = read_string(line)
        ytable[key] = val
    if line[0:5] == "g_2^3":
        key,val = read_string(line)
        ztable[key] = val
    if line[0:5] == "g_2^4":
        key,val = read_string(line)
        wtable[key] = val
        
table = {"g_2^1":xtable, "g_2^2":ytable, "g_2^3":ztable, "g_2^4":wtable}

In [4]:
# Write a function which can multiply elements in A(d) with generators (x,y,z,w) based on multiplication table
def multiplication(generator, g_dic):
    if type(g_dic) == str:
        g_dic = {g_dic:1}
    
    tab = table[generator]
    
    product = {}
    for k1, v1 in g_dic.items():
        if k1 not in tab.keys():
            pass
        else:
            eq = tab[k1]
            for k2,v2 in eq.items():
                if k2 in product.keys():
                    product[k2] += v1*v2
                else:
                    product[k2] = v1*v2
    return product

In [5]:
# Generate all monomials, will conatain dublicates such as xxy and yxx.
def gen_monoms(alphabet = "xyzw"):
    
    generators = ["g_2^1","g_2^2","g_2^3","g_2^4"]
    
    monomials = [(name,{g:1}) for name,g in zip(alphabet,generators)]

    iters = 0
    for char, generator in zip(alphabet,generators):
        for name,g in monomials:
            prod = multiplication(generator, g)
            
            if prod == {}:
                pass
            else:
                monomials.append((name+char, prod))
                iters +=1
                if iters > 1000:
                    return monomials
    return monomials

In [6]:
monoms = gen_monoms()
monoms[0:10]

[('x', {'g_2^1': 1}),
 ('y', {'g_2^2': 1}),
 ('z', {'g_2^3': 1}),
 ('w', {'g_2^4': 1}),
 ('xx', {'g_3^1': 3, 'g_2^2': 2}),
 ('yx', {'g_4^1': 2, 'g_2^1,3^1': 3, 'g_2^3': 3}),
 ('zx', {'g_2^1,4^1': 2, 'g_2^2,3^1': 3, 'g_2^4': 4}),
 ('wx', {'g_2^2,4^1': 2, 'g_2^3,3^1': 3}),
 ('xxx', {'g_4^1': 16, 'g_2^1,3^1': 9, 'g_2^3': 6}),
 ('yxx',
  {'g_5^1': 25, 'g_2^1,4^1': 20, 'g_3^2': 18, 'g_2^2,3^1': 15, 'g_2^4': 12})]

In [7]:
# Get the norm of a monomial so we can sort the equations by norm
def get_norm(monomname):
    answer = 0
    D = {c:(i+1) for i,c in enumerate("xyzw")}
    for char in monomname:
        answer += D[char]
    return answer

In [8]:
# Write functions so the output can be copy-pasted to TeX

#add curly brackets to subscript
def add_curl(string):
    return string[0:2] + "{" + (string[2:]).replace("^1","") + "}"


# Turns monomials xxy and yxx in to x^2y
def format_monomname(string):
    counts = {c:0 for c in "xyzw"}
    for char in string:
        counts[char] +=1
    s = ""
    for char in "xyzw":
        if counts[char] == 1:
            s += char
        elif counts[char]>1:
            s += char + "^" + str(counts[char])
    return s

# Output as an equation in TeX
def to_tex(name, dic):
    s = name
    s += " &= "
    for key, value in dic.items():
        s += str(value) + add_curl(key) + " + "
    return s[0:-3]+"\\\\"

In [9]:
# Sort the monomials and omit dublicates
unique_names = set([])
sorted_monoms = {i:[] for i in range(2,9)}
for name, dic in monoms[4:]:
    n = get_norm(name)
    name2 = format_monomname(name)
    if name2 in unique_names:
        pass
    else:
        unique_names.add(name2)
        sorted_monoms[n] += [(name2, dic)]

In [10]:
# Output to latex input
for i in range(2,9):
    print("\\text{norm " + str(i) + " equations} \\\\")
    for name,dic in sorted_monoms[i]:
        print(to_tex(name,dic))

\text{norm 2 equations} \\
x^2 &= 3g_{3} + 2g_{2^2}\\
\text{norm 3 equations} \\
xy &= 2g_{4} + 3g_{2,3} + 3g_{2^3}\\
x^3 &= 16g_{4} + 9g_{2,3} + 6g_{2^3}\\
\text{norm 4 equations} \\
xz &= 2g_{2,4} + 3g_{2^2,3} + 4g_{2^4}\\
x^2y &= 25g_{5} + 20g_{2,4} + 18g_{3^2} + 15g_{2^2,3} + 12g_{2^4}\\
x^4 &= 125g_{5} + 64g_{2,4} + 54g_{3^2} + 36g_{2^2,3} + 24g_{2^4}\\
y^2 &= 5g_{5} + 4g_{2,4} + 9g_{3^2} + 6g_{2^2,3} + 6g_{2^4}\\
\text{norm 5 equations} \\
xw &= 2g_{2^2,4} + 3g_{2^3,3}\\
x^2z &= 12g_{6} + 25g_{2,5} + 12g_{3,4} + 24g_{2^2,4} + 18g_{2,3^2} + 21g_{2^3,3}\\
x^3y &= 324g_{6} + 200g_{2,5} + 162g_{3,4} + 124g_{2^2,4} + 108g_{2,3^2} + 81g_{2^3,3}\\
x^5 &= 1296g_{6} + 625g_{2,5} + 480g_{3,4} + 320g_{2^2,4} + 270g_{2,3^2} + 180g_{2^3,3}\\
yz &= 3g_{6} + 5g_{2,5} + 6g_{3,4} + 6g_{2^2,4} + 9g_{2,3^2} + 9g_{2^3,3}\\
xy^2 &= 81g_{6} + 55g_{2,5} + 44g_{2^2,4} + 60g_{3,4} + 45g_{2,3^2} + 36g_{2^3,3}\\
\text{norm 6 equations} \\
x^2w &= 12g_{2,6} + 8g_{4^2} + 25g_{2^2,5} + 12g_{2,3,4}\\
x^3z &= 3

### Compute relations by finding nulspace of coefficient matrices

In [11]:
import numpy as np
from sympy import *

In [12]:
for n in range(2,9):
    key_names = set([])
    for name, dic in sorted_monoms[n]:
        for element in dic.keys():
            key_names.add(element)
    key_names = list(key_names)
    key_names.sort()
    
    A = np.zeros((len(key_names),len(sorted_monoms[n])), dtype = int)

    for i, key in enumerate(key_names):
        for j, equation in enumerate(sorted_monoms[n]):
            if key in (equation[1]).keys():
                A[i,j] = (equation[1])[key]

    print("norm", n, "variables:" ,key_names)
    #This computes the actural relations
    #pprint(Matrix(A).nullspace()) 
    
    #This computes the number of relations
    print("relations:", len(Matrix(A).nullspace())) 

norm 2 variables: ['g_2^2', 'g_3^1']
relations: 0
norm 3 variables: ['g_2^1,3^1', 'g_2^3', 'g_4^1']
relations: 0
norm 4 variables: ['g_2^1,4^1', 'g_2^2,3^1', 'g_2^4', 'g_3^2', 'g_5^1']
relations: 0
norm 5 variables: ['g_2^1,3^2', 'g_2^1,5^1', 'g_2^2,4^1', 'g_2^3,3^1', 'g_3^1,4^1', 'g_6^1']
relations: 0
norm 6 variables: ['g_2^1,3^1,4^1', 'g_2^1,6^1', 'g_2^2,5^1', 'g_3^1,5^1', 'g_3^3', 'g_4^2', 'g_7^1']
relations: 2
norm 7 variables: ['g_2^1,7^1', 'g_3^1,6^1', 'g_4^1,5^1', 'g_8^1']
relations: 7
norm 8 variables: ['g_9^1']
relations: 14


### With a bit of thinking we can extract the relations in A(d)

In [13]:
# note that some cycletypes have too large support for A(8) such as "g_2^3,3^1":
def support(s):
    answer = 0
    for substr in (s[2:]).split(","):
        answer += int(substr[0])*int(substr[2])
    return answer
support('g_2^3,3^1')

9

In [14]:
# Define a function which eliminates elements with too lage support and find relations
def relations(d=9, only_number = True):
    not_gens = "wzyx"[0:4-d//2] #Generators that vanish in lower dimension d.
    
    for n in range(2,d):
        key_names = set([])
        monom_names = []

        # Get the relevant monomials and basis elements in A(d) of norm n.
        for name, dic in sorted_monoms[n]:
            # Consider only monomials of _non-zero_ generators:
            monom_non_triv = True
            for char in name:
                if char in not_gens:
                    monom_non_triv = False
            if monom_non_triv:
                monom_names.append(name)
                # Take all summands with valid support
                for element in dic.keys():
                    if support(element) <= d: #Sort out elements with support equal to 9.
                        key_names.add(element)
        key_names = list(key_names)
        
        # Create a matrix with coefficients from the monomial equations
        A = np.zeros((len(key_names),len(sorted_monoms[n])), dtype = int)

        for i, key in enumerate(key_names):
            for j, equation in enumerate(sorted_monoms[n]):
                if key in (equation[1]).keys():
                    A[i,j] = (equation[1])[key]

        print("Norm", n, "Monomials" ,monom_names)
        if only_number:
            print("#Relations:", len(Matrix(A).nullspace())) #This computes the number of relations
        else:
            pprint(Matrix(A).nullspace())
            
relations()

Norm 2 Monomials ['x^2']
#Relations: 0
Norm 3 Monomials ['xy', 'x^3']
#Relations: 0
Norm 4 Monomials ['xz', 'x^2y', 'x^4', 'y^2']
#Relations: 0
Norm 5 Monomials ['xw', 'x^2z', 'x^3y', 'x^5', 'yz', 'xy^2']
#Relations: 0
Norm 6 Monomials ['x^2w', 'x^3z', 'x^4y', 'x^6', 'yw', 'xyz', 'x^2y^2', 'y^3', 'z^2']
#Relations: 2
Norm 7 Monomials ['x^3w', 'x^4z', 'x^5y', 'x^7', 'xyw', 'x^2yz', 'x^3y^2', 'y^2z', 'xy^3', 'zw', 'xz^2']
#Relations: 7
Norm 8 Monomials ['x^4w', 'x^5z', 'x^6y', 'x^8', 'x^2yw', 'x^3yz', 'x^4y^2', 'y^2w', 'xy^2z', 'x^2y^3', 'y^4', 'xzw', 'x^2z^2', 'yz^2', 'w^2']
#Relations: 14


In [15]:
relations(6,False)

Norm 2 Monomials ['x^2']
[]
Norm 3 Monomials ['xy', 'x^3']
[]
Norm 4 Monomials ['xz', 'x^2y', 'x^4', 'y^2']
⎡⎡ 27 ⎤⎤
⎢⎢ ── ⎥⎥
⎢⎢ 10 ⎥⎥
⎢⎢    ⎥⎥
⎢⎢-19 ⎥⎥
⎢⎢────⎥⎥
⎢⎢ 20 ⎥⎥
⎢⎢    ⎥⎥
⎢⎢3/20⎥⎥
⎢⎢    ⎥⎥
⎣⎣ 1  ⎦⎦
Norm 5 Monomials ['x^2z', 'x^3y', 'x^5', 'yz', 'xy^2']
⎡⎡1⎤  ⎡ 0 ⎤  ⎡ 0  ⎤  ⎡ 0  ⎤  ⎡  0  ⎤⎤
⎢⎢ ⎥  ⎢   ⎥  ⎢    ⎥  ⎢    ⎥  ⎢     ⎥⎥
⎢⎢0⎥  ⎢-27⎥  ⎢-108⎥  ⎢-1/4⎥  ⎢-27/4⎥⎥
⎢⎢ ⎥  ⎢   ⎥  ⎢    ⎥  ⎢    ⎥  ⎢     ⎥⎥
⎢⎢0⎥  ⎢ 1 ⎥  ⎢ 0  ⎥  ⎢ 0  ⎥  ⎢  0  ⎥⎥
⎢⎢ ⎥, ⎢   ⎥, ⎢    ⎥, ⎢    ⎥, ⎢     ⎥⎥
⎢⎢0⎥  ⎢ 0 ⎥  ⎢ 1  ⎥  ⎢ 0  ⎥  ⎢  0  ⎥⎥
⎢⎢ ⎥  ⎢   ⎥  ⎢    ⎥  ⎢    ⎥  ⎢     ⎥⎥
⎢⎢0⎥  ⎢ 0 ⎥  ⎢ 0  ⎥  ⎢ 1  ⎥  ⎢  0  ⎥⎥
⎢⎢ ⎥  ⎢   ⎥  ⎢    ⎥  ⎢    ⎥  ⎢     ⎥⎥
⎣⎣0⎦  ⎣ 0 ⎦  ⎣ 0  ⎦  ⎣ 0  ⎦  ⎣  1  ⎦⎦
